In [13]:
import pandas as pd
import sys
sys.path.append("/Users/yuksehyun/Desktop/NH나무/FinNLP")
from finnlp.data_sources.news.finnhub_date_range import Finnhub_Date_Range
from finnlp.data_sources.news.yahoo_streaming import Yahoo_Date_Range
from datasets import load_dataset

STK_QUT = pd.read_csv('STK_QUT.csv')
STK_IEM = pd.read_csv('STK_IEM.csv',encoding='EUC-KR')

data = list(STK_IEM['tck_iem_cd'].unique())
#name data
name_data = list(STK_IEM['fc_sec_eng_nm'].unique())
named_data = [ x.split() for x in name_data]
name_data = [ " ".join(x) for x in named_data]

#ticker data
ticker_data = [ x.split()[0] for x in data]

In [16]:
data = load_dataset('sehyun66/Finnhub-News',split='train')

Found cached dataset parquet (/Users/yuksehyun/.cache/huggingface/datasets/sehyun66___parquet/sehyun66--Finnhub-News-ae5d34837438db29/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


In [26]:
df=pd.DataFrame(data)

KeyboardInterrupt: 

In [ ]:
df[df['related']=='ANGH']

In [ ]:
import matplotlib.pyplot as plt

y = [len(df[df['related']==ticker]) for ticker in ticker_data]
x = list(range(len(ticker_data)))

# Create a histogram

In [ ]:
plt.bar(x, y)

# Label the axes and give a title
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Histogram with X as Index and Y as Values')
plt.ylim((0,2000))
# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import mplfinance as fplt

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot()  
mpl_finance.candlestick2_ohlc(ax, Nvidia['gts_iem_ong_pr'], Nvidia['gts_iem_hi_pr'], Nvidia['gts_iem_low_pr'], Nvidia['gts_iem_end_pr'], width=0.5, colorup='r', colordown='b')
ax.xaxis.set_ticks([n for n in range(1,len(TimeIndex),30)])
ax.set_xticklabels([TimeIndex.iloc[n] for n in range(1,len(TimeIndex),30)])
plt.show()

In [ ]:
fplt.plot(  data,
            type='candle',
            style='charles',
            title='Nvidia, March - 2020',
            ylabel='Price ($)',
            volume=True,
            ylabel_lower='Shares\nTraded',
        )

In [140]:
Nvidia.index = TimeIndex
data = Nvidia.drop(columns=['trd_dt','tck_iem_cd'])
data.index.name = 'Date'
data.columns = ['Open','High','Low','Close','Volume','sll','byn']
data.index = pd.DatetimeIndex(data.index)